### Step 1 - Start Spark Session and Include additional configurations and common functions

In [1]:
%run "../includes/configurations"

In [2]:
%run "../includes/common_functions"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize a Spark session
spark = SparkSession.builder.appName("PitStopsIngestion").getOrCreate()

23/12/29 11:36:07 WARN Utils: Your hostname, falcao-sys resolves to a loopback address: 127.0.1.1; using 192.168.11.185 instead (on interface wlx7898e8c12476)
23/12/29 11:36:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 11:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2 - Read the JSON file sing the Spark Dataframe Reader

In [4]:
pit_stops_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("stop", StringType(), False),
    StructField("lap", IntegerType(), False),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), False),
    StructField("milliseconds", IntegerType(), True)
])

In [5]:
pit_stops_df = spark.read.schema(pit_stops_schema).option("multiLine", True).json("../data/pit_stops.json")

### Step 3 - Rename and add new columns

In [6]:
pit_stops_df = pit_stops_df\
    .withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("driverId", "driver_id")

In [7]:
pit_stops_df = add_ingestion_date(pit_stops_df)

### Step 4 - Write to output to process container in Parquet Format

In [8]:
pit_stops_df.write.mode("overwrite").parquet("../processed_data/pit_stops")